<a href="https://colab.research.google.com/github/styopochkin23/data-analytics-portfolio/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# 1. Підключаємо Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Імпорти
import pandas as pd
import numpy as np
from scipy.stats import norm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
file_path = "/content/drive/MyDrive/projects/Final Project/Portfolio2.csv"
df = pd.read_csv(file_path)

In [30]:
df.columns


Index(['date', 'country', 'device', 'continent', 'channel', 'test',
       'test_group', 'event_name', 'value'],
      dtype='object')

In [31]:
# Фіксуємо 4 метрики для A/B тесту
metrics = [
    "add_payment_info",
    "add_shipping_info",
    "begin_checkout",
    "new_account"
]

df['event_name'].value_counts()


,count
event_name,
session,107210
session_start,106242
page_view,101907
user_engagement,94520
first_visit,81621
scroll,73643
view_promotion,61695
view_item,44869
session_with_orders,25892


In [32]:
# 1. Фіксуємо метрики
metrics = [
    "add_payment_info",
    "add_shipping_info",
    "begin_checkout",
    "new_account"
]

# 2. Універсальна функція розрахунку
def calculate_ab_stats(df, group_cols, metrics):
    df_agg = df.groupby(group_cols + ['test_group', 'event_name'])['value'].sum().reset_index()
    unique_groups = df_agg[group_cols].drop_duplicates()
    results = []

    for _, group_row in unique_groups.iterrows():
        mask = pd.Series([True] * len(df_agg))
        for col in group_cols:
            mask &= (df_agg[col] == group_row[col])
        current_data = df_agg[mask]

        co_data = current_data[current_data['test_group'] == 1]
        ev_data = current_data[current_data['test_group'] == 2]

        try:
            co_sess = co_data[co_data['event_name'] == 'session']['value'].values[0]
            ev_sess = ev_data[ev_data['event_name'] == 'session']['value'].values[0]
        except IndexError: continue

        if co_sess == 0 or ev_sess == 0: continue

        for metric in metrics:
            co_count = co_data[co_data['event_name'] == metric]['value'].sum()
            ev_count = ev_data[ev_data['event_name'] == metric]['value'].sum()

            p_co, p_ev = co_count / co_sess, ev_count / ev_sess
            p_pool = (co_count + ev_count) / (co_sess + ev_sess)
            se = np.sqrt(p_pool * (1 - p_pool) * (1/co_sess + 1/ev_sess))

            if se == 0: z_stat, p_val = 0, 1.0
            else:
                z_stat = (p_ev - p_co) / se
                p_val = 2 * (1 - norm.cdf(abs(z_stat)))

            row = {col: group_row[col] for col in group_cols}
            row.update({
                'segment_type': ' + '.join(group_cols),
                'metric': metric,
                'users_control': co_sess, 'users_test': ev_sess,
                'conv_control': p_co, 'conv_test': p_ev,
                'lift_pct': ((p_ev - p_co) / p_co * 100) if p_co != 0 else 0,
                'z_stat': z_stat, 'p_value': p_val,
                'significant': p_val < 0.05
            })
            results.append(row)

    return pd.DataFrame(results)

# 3. Запуск розрахунків
print("Рахуємо статистику...")
df_test = calculate_ab_stats(df, ['test'], metrics)
df_country = calculate_ab_stats(df, ['test', 'country'], metrics)
df_device = calculate_ab_stats(df, ['test', 'device'], metrics)

# 4. Об'єднання
final_df = pd.concat([df_test, df_country, df_device], ignore_index=True)
final_df = final_df.fillna(0)
final_df['significant'] = final_df['significant'].fillna(False)

# 5. ЗБЕРЕЖЕННЯ
output_path = "/content/drive/MyDrive/projects/Final Project/ab_test_tableau_ready.csv"
final_df.to_csv(output_path, index=False)

print(f"✅ Готово!: {output_path}")
print(final_df.head())

Рахуємо статистику...


/tmp/ipython-input-2288705104.py:37: RuntimeWarning: invalid value encountered in sqrt
  se = np.sqrt(p_pool * (1 - p_pool) * (1/co_sess + 1/ev_sess))


✅ Готово!: /content/drive/MyDrive/projects/Final Project/ab_test_tableau_ready.csv
   test segment_type             metric  users_control  users_test  \
0     1         test   add_payment_info          45362       45193   
1     1         test  add_shipping_info          45362       45193   
2     1         test     begin_checkout          45362       45193   
3     1         test        new_account          45362       45193   
4     2         test   add_payment_info          50637       50244   

   conv_control  conv_test   lift_pct    z_stat   p_value  significant  \
0      0.043825   0.049322  12.542021  3.924884  0.000087         True   
1      0.066884   0.071272   6.560481  2.603571  0.009226         True   
2      0.083418   0.088974   6.660587  2.978783  0.002894         True   
3      0.000000   0.000000   0.000000  0.000000  1.000000        False   
4      0.046290   0.047946   3.576911  1.240994  0.214608        False   

  country device  
0       0      0  
1       0    

# A/B Test Analysis – Conversion Funnel Optimization

This project analyzes the impact of multiple A/B tests on key conversion funnel events using statistical hypothesis testing.

The goal is to:
- Evaluate whether product changes led to statistically significant improvements
- Measure conversion lift across critical user journey steps
- Support product decision-making with data-driven evidence

Key funnel events analyzed:
- add_payment_info  
- add_shipping_info  
- begin_checkout  
- new_account


### Data Source

The dataset contains aggregated session and event data split by:
- test number
- experimental group (EV)
- control group (CO)

Each test includes session counts and event occurrences required to compute conversion rates and statistical significance.

Original dataset:
[📎 https://colab.research.google.com/drive/1J_TcfI0g4hz4yQ9vwPUBSTCVa5q5H1un?usp=drive_link]  ⬅️


### A/B Testing Methodology

For each metric, the following steps were applied:

1. Calculation of conversion rates:
   - EV = event_count / session_count
   - CO = event_count / session_count

2. Statistical hypothesis testing:
   - Two-sided Z-test for proportions
   - Pooled probability used for variance estimation

3. Statistical significance:
   - Confidence level: 95%
   - Significance threshold: p-value < 0.05

4. Output metrics:
   - Z-statistic
   - P-value
   - Statistical significance flag


### Final A/B Test Results

The calculated results are stored in a separate CSV file:

📄 A/B Test Results File:
[📎 https://drive.google.com/file/d/1xthNicn9jFHRs0tg0hJvxQsoa3XY_acA/view?usp=drive_link]  

The file contains the following fields:
- test_number
- metric
- conversion_rate_ev
- conversion_rate_co
- z_stat
- p_value
- significant


### Interactive Dashboard (Tableau)

An interactive Tableau dashboard was built to visualize:
- Conversion performance across funnel steps
- Statistical significance of test results
- Comparison across multiple A/B tests

📊 Tableau Dashboard:
[📎https://public.tableau.com/app/profile/oleksandr.stopochkin/viz/Portfolio2_17649401032260/Dashboard1?publish=yes]  


## Conclusions

1. Several funnel steps demonstrated statistically significant changes (p < 0.05), indicating real behavioral impact from product experiments.

2. The strongest effects were observed on intermediate funnel steps such as:
   - add_payment_info
   - begin_checkout

3. The final step (new_account) showed no statistically significant change across tests, indicating potential downstream conversion bottlenecks.

4. The analysis confirms that:
   - Not every conversion lift is statistically reliable
   - Product decisions should rely on statistical validation rather than raw percentage changes

This project demonstrates a full-cycle product A/B testing workflow:
Data → Statistical Testing → Visualization → Business Interpretation
